# Script to download and combine DECam bricks NEW VERSION
8 Feb 2025 version (J. Jensen, E. Kourkchi)

In [1]:
import os, sys
import astropy.io.fits as fits
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab as py

pysbf_path = "/Users/Joe/data/sbf/"
sys.path.insert(0, pysbf_path)
from pysbf import *


In [2]:
def add_hdr(hdr, header, key):
    
    value = header[key]
    
    if isinstance(value, str):
        value = "'" + value + "'"
    else:
        value = str(value)
    
    hdr += key + "=" + value + " / " + header.comments[key] + "\n"
    
    return hdr

In [3]:
def load_info(galaxy_name, default_size = 0):
    
    # import pdb; pdb.set_trace()
    
    ned = get_ned_info([galaxy_name])
    
    try:
        if ned is not None:
            objName = ned.iloc[0]["Object Name"]
            ra, dec = ned.iloc[0]["ra_dec"]

            try:
                pgc = get_ned_PGC(objName.replace(" ", "%20"))
            except:
                pgc = "None"
                
            try:
                logd25 = Leda.loc[pgc]["logd25"]
            except: 
                logd25 = default_size

            # For some reason this galaxy didn't work with the coords request from NED
            if galaxy_name == "ic2597":
                ra = 159.447716
                dec = -27.081695
            # The NED coordinates for these are between the main galaxy and a satellite    
            if galaxy_name == "n7242":
                ra = 333.91436
                dec = 37.2985915
            if galaxy_name == "n5222":
                ra = 203.7330204
                dec = 13.7420058
            if galaxy_name == "n1684":
                ra = 73.1297411
                dec = -3.1060132
            if galaxy_name == "n0997":
                ra = 39.3102843
                dec = 7.3056710
            if galaxy_name == "n7187":
                ra = 330.6854069
                dec = -32.8031856
                
            return pgc, objName.replace(" ", ""), ra, dec, logd25
    except: 
        print(f"Couldn't load data for {galaxy_name}. Using default image size.")
        pass
        
    return None

In [4]:
def get_decam(DECam_data, bricks, pgc, objName, ra, dec, logd25, filters):

    #import pdb; pdb.set_trace()
    
    temp_repo_name = DECam_data + objName
    d25 = 10**logd25
    size = 0.1 * d25 / 60  # deg
    delta = size * 8
    pix_size = 0.262   # arcsec
    naxis = int(8*size*3600 / pix_size)
    
    if naxis > 5000:  # If the image is really big, just use max size of 5000 pix
        naxis=5000    # pixels for cutout
        delta = 0.365   # deg (5000 pix = 0.365 deg)
    if naxis < 1500 or d25==0:  # If the image is small, just use min size of 1500 pix
        naxis=1500    # pixels for cutout
        delta = 0.109   # deg (1500 pix = 0.109 deg)
    print('naxis = ',naxis)

    ra_max  =  ra+delta
    ra_min  =  ra-delta
    dec_max =  dec+delta
    dec_min =  dec-delta


    dg = bricks[(bricks["RA1"]<ra_max) & 
       (bricks["RA2"]>ra_min) &
       (bricks["DEC1"]<dec_max) & 
       (bricks["DEC2"]>dec_min) 
      ]

    if not os.path.isdir(temp_repo_name):
        os.mkdir(temp_repo_name)

    for filt in filters:
        urls = [] 
        for iid in dg.BRICKNAME.values:

            s = str(iid)[2:-1]

            base_url = f"https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/{s[:3]}/{s}/"
            image_name = f"legacysurvey-{s}-image-{filt}.fits.fz"
            url = base_url + image_name
            urls.append(url)
 
        with cwd(temp_repo_name):

            for url in urls:
                xcmd(f"wget {url}", verbose=False)

            cmd = 'mkdir tmp'
            xcmd(cmd, False)

            for url in urls:

                fname = url.split("/")[-1].split(".")[0]
                hdulist = fits.open(fname+".fits.fz")
                prihdr = hdulist[1].header 

                fits.writeto('tmp/' + fname+'.fits', hdulist[1].data, header=hdulist[1].header, 
                                        output_verify='exception', overwrite=True, checksum=False)

                fout=open('montage.csh','w')

            montage = '''
              rm *tbl
              rm -rf projected
              mkdir  projected
              mImgtbl tmp rimages.tbl
              mProjExec -p tmp rimages.tbl tmp.hdr projected stats.tbl
              mImgtbl projected pimages.tbl
              mAdd -p projected pimages.tbl tmp.hdr tmp.fits
              mConvert -b -32 tmp.fits tmp.fits
              rm *area*fits
              rm *tbl
              rm -rf projected
              '''

            fout.write(montage+'\n')
            fout.close()

            crpix = naxis/2
            hdr = ''
            hdr += 'SIMPLE  = T' + '\n'
            hdr += 'BITPIX  = -32' + '\n'
            hdr += 'NAXIS   = 2' + '\n'
            hdr += 'NAXIS1  = '+str(naxis) + '\n'
            hdr += 'NAXIS2  = '+str(naxis) + '\n'
            hdr += 'CTYPE1  = \'RA---TAN\'' + '\n'
            hdr += 'CTYPE2  = \'DEC--TAN\'' + '\n'
            hdr += 'CRVAL1  = ' +str(ra)+ '\n'
            hdr += 'CRVAL2  = ' +str(dec)+ '\n'
            hdr += 'CRPIX1  = '+str(crpix) + '\n'
            hdr += 'CRPIX2  = '+str(crpix) + '\n'
            hdr += 'CDELT1  = -7.27777778E-05' + '\n'
            hdr += 'CDELT2  = 7.27777778E-05' + '\n'
            hdr += 'CROTA2  = 0.000000' + '\n'
            hdr += 'EQUINOX = 2000.0' + '\n'
            hdr += 'BSCALE  = 1' + '\n'
            hdr += 'BZERO   = 0' + '\n'
            hdr += 'EXPTIME = 1.0' + '\n'
            hdr += 'ZP      = 16.40006562 ' + '\n'
            hdr = add_hdr(hdr, prihdr, "MAGZERO")
            hdr = add_hdr(hdr, prihdr, "BUNIT")
            hdr = add_hdr(hdr, prihdr, "COSKY_"+filt.upper())
            hdr = add_hdr(hdr, prihdr, "FILTER")
            hdr = add_hdr(hdr, prihdr, "FILTERX")
            hdr = add_hdr(hdr, prihdr, "INSTRUME")
            hdr = add_hdr(hdr, prihdr, "TELESCOP")
            hdr = add_hdr(hdr, prihdr, "OBSERVAT")
            hdr = add_hdr(hdr, prihdr, "EQUINOX")
            hdr = add_hdr(hdr, prihdr, "LSDR")
            hdr = add_hdr(hdr, prihdr, "SURVEY")
            hdr = add_hdr(hdr, prihdr, "SURVEYID")
            hdr = add_hdr(hdr, prihdr, "LEGPIPEV")
        #     hdr = add_hdr(hdr, prihdr, "")
        #     hdr = add_hdr(hdr, prihdr, "")

            hdr += 'PGC = ' + 'PGC'+str(pgc) + '\n'
            hdr += 'OBJECT = ' + objName + '\n'
            hdr += 'HISTORY = \'email: <ehsan20@hawaii.edu>\'' + '\n'  
            hdr += 'END' + '\n'

            fout=open('tmp.hdr','w')
            fout.write(hdr)
            fout.close()

            xcmd('csh montage.csh', False)
            xcmd('rm montage.csh', False)
            xcmd('rm tmp.hdr', False)
            xcmd('rm -rf tmp', False)
            xcmd('rm -rf *.fits.fz*', False)
            xcmd(f'mv tmp.fits {objName}_{filt}.fits', True)

In [5]:
def get_decvar(DECam_data, bricks, pgc, objName, ra, dec, logd25, filters):

    temp_repo_name = DECam_data + objName
    d25 = 10**logd25
    size = 0.1 * d25 / 60  # deg
    delta = size * 8
    pix_size = 0.262   # arcsec
    naxis = int(8*size*3600 / pix_size)
    
    if naxis > 5000:  # If the image is really big, just use max size of 5000 pix
        naxis=5000    # pixels for cutout
        delta = 0.365   # deg (5000 pix = 0.365 deg)
    if naxis < 1500 or d25==0:  # If the image is small, just use min size of 1500 pix
        naxis=1500    # pixels for cutout
        delta = 0.109   # deg (1500 pix = 0.109 deg)
    print('naxis = ',naxis)

    ra_max  =  ra+delta
    ra_min  =  ra-delta
    dec_max =  dec+delta
    dec_min =  dec-delta

    dg = bricks[(bricks["RA1"]<ra_max) & 
       (bricks["RA2"]>ra_min) &
       (bricks["DEC1"]<dec_max) & 
       (bricks["DEC2"]>dec_min) 
      ]

    if not os.path.isdir(temp_repo_name):
        os.mkdir(temp_repo_name)

    for filt in filters:
        urls = [] 
        for iid in dg.BRICKNAME.values:

            s = str(iid)[2:-1]
            
            base_url = f"https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/{s[:3]}/{s}/"
            var_name = f"legacysurvey-{s}-invvar-{filt}.fits.fz"
            url = base_url + var_name
            urls.append(url)

        with cwd(temp_repo_name):

            for url in urls:
                xcmd(f"wget {url}", verbose=False)

            cmd = 'mkdir tmpvar'
            xcmd(cmd, False)

            for url in urls:

                fname = url.split("/")[-1].split(".")[0]
                hdulist = fits.open(fname+".fits.fz")
                prihdr = hdulist[1].header 

                fits.writeto('tmpvar/' + fname+'.fits', hdulist[1].data, header=hdulist[1].header, 
                                        output_verify='exception', overwrite=True, checksum=False)

                # Invert the invvar image using monsta (for convenience)
                monsta_script = """
                    rd 1 'tmpvar/"""+fname+""".fits'
                    inverse 1
                    wd 1 'tmpvar/"""+fname+""".fits'
                """
                run_monsta(monsta_script, 'monsta.pro', 'monsta.log')
                !cat monsta.log
                
                fout=open('montage.csh','w')

            montage = '''
              rm *tbl
              rm -rf projectedvar
              mkdir  projectedvar
              mImgtbl tmpvar varimages.tbl
              mProjExec -p tmpvar varimages.tbl tmpvar.hdr projectedvar varstats.tbl
              mImgtbl projectedvar pvarimages.tbl
              mAdd -p projectedvar pvarimages.tbl tmpvar.hdr tmpvar.fits
              mConvert -b -32 tmpvar.fits tmpvar.fits
              rm *area*fits
              rm *tbl
              rm -rf projectedvar
              '''

            fout.write(montage+'\n')
            fout.close()

            crpix = naxis/2
            hdr = ''
            hdr += 'SIMPLE  = T' + '\n'
            hdr += 'BITPIX  = -32' + '\n'
            hdr += 'NAXIS   = 2' + '\n'
            hdr += 'NAXIS1  = '+str(naxis) + '\n'
            hdr += 'NAXIS2  = '+str(naxis) + '\n'
            hdr += 'CTYPE1  = \'RA---TAN\'' + '\n'
            hdr += 'CTYPE2  = \'DEC--TAN\'' + '\n'
            hdr += 'CRVAL1  = ' +str(ra)+ '\n'
            hdr += 'CRVAL2  = ' +str(dec)+ '\n'
            hdr += 'CRPIX1  = '+str(crpix) + '\n'
            hdr += 'CRPIX2  = '+str(crpix) + '\n'
            hdr += 'CD1_1   = -7.27777778E-05' + '\n'
            hdr += 'CD2_2   = 7.27777778E-05' + '\n'
            hdr += 'CROTA2  = 0.000000' + '\n'
            hdr += 'EQUINOX = 2000.0' + '\n'
            hdr += 'BSCALE  = 1' + '\n'
            hdr += 'BZERO   = 0' + '\n'
       #     hdr += 'EXPTIME = 1.0' + '\n'
       #     hdr += 'ZP      = 16.40006562 ' + '\n'
       #     hdr = add_hdr(hdr, prihdr, "MAGZERO")
            hdr = add_hdr(hdr, prihdr, "BUNIT")
       #     hdr = add_hdr(hdr, prihdr, "COSKY_"+filt.upper())
            hdr = add_hdr(hdr, prihdr, "FILTER")
            hdr = add_hdr(hdr, prihdr, "FILTERX")
            hdr = add_hdr(hdr, prihdr, "INSTRUME")
            hdr = add_hdr(hdr, prihdr, "TELESCOP")
            hdr = add_hdr(hdr, prihdr, "OBSERVAT")
            hdr = add_hdr(hdr, prihdr, "EQUINOX")
            hdr = add_hdr(hdr, prihdr, "LSDR")
            hdr = add_hdr(hdr, prihdr, "SURVEY")
            hdr = add_hdr(hdr, prihdr, "SURVEYID")
            hdr = add_hdr(hdr, prihdr, "LEGPIPEV")
        #     hdr = add_hdr(hdr, prihdr, "")
        #     hdr = add_hdr(hdr, prihdr, "")

            hdr += 'PGC = ' + 'PGC'+str(pgc) + '\n'
            hdr += 'OBJECT = ' + objName + '\n'
            hdr += 'HISTORY = \'email: <jjensen@uvu.edu>\'' + '\n'  
            hdr += 'END' + '\n'

            fout=open('tmpvar.hdr','w')
            fout.write(hdr)
            fout.close()

            # Multiply the VAR images by an arbitrary scale factor (experimental)
            #monsta_script = """
            #    rd 1 'tmpvar/tmpvar.fits'
            #    mc 1 1.
            #    wd 1 'tmpvar/tmpvar.fits'
            #    """
            #run_monsta(monsta_script, 'monsta.pro', 'monsta.log')
            #!cat monsta.log

            xcmd('csh montage.csh', False)
            xcmd('rm montage.csh', False)
            xcmd('rm tmpvar.hdr', False)
            xcmd('rm -rf tmpvar', False)
            xcmd('rm -rf *.fits.fz*', False)
            xcmd(f'mv tmpvar.fits {objName}_{filt}var.fits', True)
                     

In [6]:
from astropy.table import Table

data = Table.read('survey-bricks.fits', format='fits')
bricks = data.to_pandas()

In [7]:
filters = ["g","z","i"]

# ACS color calibration galaxies:
# DECam_data = "/Users/Joe/data/DECam/"
# galaxies = [
#"ic0798",
#"ic0809",
#"ic3019",
#"ic3025",
#"ic3032",
#"ic3065",
#"ic3101",
#"ic3292",
#"ic3328",
#"ic3381",
#"ic3383",
#"ic3442",
#"ic3461",
#"ic3468",
#"ic3470",
#"ic3487",
#"ic3490",
#"ic3492",
#"ic3501",
#"ic3509",
#"ic3586",
#"ic3602",
#"ic3612",
#"ic3633",
#"ic3635",
#"ic3647",
#"ic3652",
#"ic3653",
#"ic3735",
#"ic3773",
#"ic3779",
#"n4262",
#"n4267",
#"n4309a",
#"n4318",
#"n4340",
#"n4350",
#"n4352",
#"n4371",
#"n4377",
#"n4379",
#"n4387",
#"n4417",
#"n4434",
#"n4435",
#"n4452",
#"n4458",
#"n4464",
#"n4467",
#"n4472",
#"n4474",
#"n4476",
#"n4478",
#"n4479",
#"n4482",
#"n4483",
#"n4486a",
#"n4486b",
#"n4489",
#"n4515",
#"n4528",
#"n4550",
#"n4551",
#"n4564",
#"n4570",
#"n4578",
#"n4612",
#"n4623",
#"n4638",
#"n4649",
#"n4660",
#"u7399a",
#"u7436",
#"u7580",
#"u7854",
#"n4874",
#"n4993",
#"ic1919",
#"ic2006",
#"n1344",
#"n1374",
#"n1375",
#"n1380",
#"n1399",
#"n1404",

# JWST TRGB targets (3055)
#DECam_data = "/Users/Joe/data/DECam/"
#galaxies = [
#"n4552","n4697",
#"n4374","n4406","n4621","n4486","n1549","n3379",
    
# MASSIVE + SN hosts 2021
# DECam_data = "/Users/Joe/data/DECam/"
# galaxies = [
# "ic2597",
# "n0057",
# "n0315", # no i or z data
# "n0383", # no i or z data
# "n0410", # no i or z data
# "n0495", # no i or z data
# "n0507", # incomplete g data, no i or z
# "n0524"]; filters=["g","z"] # no i
# "n0533",
# "n0545",
# "n0547",
# "n0665", # no i or z data
# "n0708", # no data
# "n0741", # incomplete i and z data
# "n0777", # no i or z data
# "n0809",
# "n0890", # no data
# "n0910", # no data
# "n1016",
# "n1060", # no data
# "n1129", # no data
# "n1167", # no data
# "n1200",
# "n1201",
# "n1259", # no data
# "n1272", # no data
# "n1278", # no data
# "n1453",
# "n1573", # no data
# "n1600",
# "n1684",
# "n1700",
# "n2258", # no data
# "n2274", # no data
# "n2340", # no data
# "n2513",
# "n2672",
# "n2693", # no data
# "n2765",
# "n2962",
# "n3158", # no data
# "n3392", # no data
# "n3504",
# "n3842",
# "n4036", # no data
# "n4073",
# "n4386", # no data
# "n4839",
# "n4874",
# "n4914", # no data
# "n4993",
# "n5322", # no data
# "n5353", # no data
# "n5490",
# "n5557",# no data
# "n5839",
# "n6482", # no i or z-band data
# "n6702", # no data
# "n6964",
# "n7052", # no data
# "n7242", # no data
# "n7619", # no i or z-band data
# "eso125-g006", # no data

# -------SN-31 galaxies (17446)
# DECam_data = "/Users/Joe/data/wfc3-17446/DECam/" 
# galaxies = [
# "cgcg-005-038",
# "cgcg-031-049"]; filters=["g","z"] # no i 
# "cgcg-285-013", # no g-band images
# "eso352-g057",
# "eso442-g015",
# "eso479-g007",
# "ic0511", # no g-band images
# "mcg-02-33-017",
# "cgcg-097-050",
# "mcg+08-07-008", # no DECam data
# "u00402",
# "n0083"]; filters=["g","z"] # no i 
# "n1209",
# "n3332",
# "n3643",
# "n3941", # no g images.
# "n4125", # no g images.
# "n4169",
# "n4415",
# "n4636",
# "n4767",
# "n5018",
# "n5222",]
# "n5304",
# "n5419",
# "n5631", # no g images
# "n7187",
# "leda-1693718", # No DECam images
# "u02829" ]; filters=["g","z"] # no i
    
# -------SNAP-28 galaxies (16262)
# DECam_data = "/Users/Joe/data/wfc3-16262/DECam/"
# galaxies = [
# "WISEAJ192159.44-600234.4", # aka 2MASS-J19215490-6003146 # 404 error
# "WISEAJ125011.54-411315.5", # aka "2MASS-J12501152-4113155"
# "eso137-g006", # no DECam files found
# "eso137-g008", # no images
# "eso137-g024", # no images
# "eso185-g054",
# "eso235-g049",
# "eso399-g010", # no i, z images
# "eso436-g045",
# "eso507-g025",
# "ic4727", # no z image 
# "ic4794",
# "ic4798",
# "ic4926", # no z image
# "ic4931", # no z image
# "ic4956",
# "ic4991", # no images
# "ic5105",
# "ic5193", # no g images
# "n0439",
# "n1490",
# "n3250", # no images
# "n3268", # no g images
# "n3283", # no data
# "n4373",    
# "n5090", # no data 
# "n6407", # no data or no z data
# "n6653",    
# "n6739", # bad stack, but we will make do
# "n6768",
# "n6841", # no z images
# "n6861",
# "n6868",
# "n7014",
# "pgc63905",
# "cgcg328-014", # no images
# "cgcg539-126", # no images
# "cgcg-540-079", # no images
# "eso461-g007", # no z data
# "eso462-g015", # no z data
# "ic5193", # no DECam data
# "n0080"]; filters=["g","z"] # no i
# "n0380"]; filters=["g","z"] # no i
# "n0679", # no DECam data
# "n0750"]; filters=["g","z"] # no i
# "n2208", # no g images.
# "n2256", # no DECam images
# "n2329", # no g images
# "n2418"]; filters=["g","z"] # no i
# "n2569"]; filters=["g","z"] # i-band image is empty
# "n3070",
# "n3091",
# "n3308",
# "n3311",
# "n4825",
# "n4955",
# "n6223", # no g data
# "n6577"]; filters=["g","z"]
# "n6688", # no g data
# "n6968",
# "n7265", # no data
# "n7274", # no data
# "n7426", # no data
# "n7618", # no data
# "pgc158229",
# "pgc170207",
# "u03353", # no data
# "u03396", # no data
# "u11990", # no data
# "u12517", # no data

#-------SNAP-31 galaxies (17436)
# DECam_data = "/Users/Joe/data/wfc3-17436/DECam/"
# galaxies = [
# "n0050",
# "n0194",
# "n0193",
# "n0277",
# "n0311", # no i or z-band images
# "n0393", # no images
# "n0499", # no i or z-band images
# "n0508", # no i or z-band images
# "n0529", # no images
# "n0541",
# "n0564",
# "n0759", # no images
# "n0883",
# "u01859", # no images
# "ic1812",
# "n1057", # no images
# "n1066", # no images
# "ic0265", # no images
# "ic0310", # no images
# "n1270", # no images
# "n1550",
# "n1609",
# "n1682",
# "n1710",
# "u03215", 
# "n1713",
# "u03683", # no images
# "n2653", # No g-band images
# "ic2437", # no data
# "n4782",
# "n5357",
# "wkk6479", # no data
# "pgc165984", # same target as wkk6479
# "n6502", # g and i, but no z data
# "ic4842",
# "u00902", # no DECam images
# "n0641",
# "eso197-g010",
# "eso153-g003",
# "n0997", # no i or z data
# "n1226", # no data
# "n1322",
# "2MASX-J04002709+3854173", # probably no images
# "n1497", # no images
# "2MASX-J04194579+3530344", # probably no images
# "u03021", # no images
# "n2340", # no images
# "u03725", # no images
# "mcg-01-23-019",
# "n3222",
# "n3352",
# "n3771",
# "n3172", # No DECam images.
# "2MASX-J11542038-5146519", # no images
# "ic1153", # No g-band images
# "n6375"]; filters=["g","z"] # no i-band images
# "n6442"]; filters=["g","z"] # no i-band images
# "pgc61702", # No images. aka ESO280-G007
# "eso141-g003",
# "ic5180", # no images
# "n7315", # no images
# "u12179"]; filters=["g","z"] # no i-band images
# "cgcg551-015", # no images
# "ic0574",
# "n3343", # No g-band images
# "ic0642",
# "n0071"]; filters=["g","z"] # no i-band images
# "n3426",
# "ic1143", # No g-band images
# "u10918", # no good data

]

In [8]:
print(filters)

['g', 'z', 'i']


In [9]:
for galaxy in galaxies:

    pgc, objName, ra, dec, logd25 = load_info(galaxy) 
    print(objName,pgc)
    get_decam(DECam_data, bricks, pgc, galaxy, ra, dec, logd25, filters)
    get_decvar(DECam_data, bricks, pgc, galaxy, ra, dec, logd25, filters)

UGC07399A 39857
naxis =  1500


--2025-02-09 20:37:35--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/185/1851p175/legacysurvey-1851p175-image-g.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.111, 128.55.206.106, 128.55.206.107, ...
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12268800 (12M)
Saving to: 'legacysurvey-1851p175-image-g.fits.fz'

     0K .......... .......... .......... .......... ..........  0%  684K 17s
    50K .......... .......... .......... .......... ..........  0% 1.34M 13s
   100K .......... .......... .......... .......... ..........  1% 10.8M 9s
   150K .......... .......... .......... .......... ..........  1%  732K 11s
   200K .......... .......... .......... .......... ..........  2%  111M 9s
   250K .......... .......... .......... .......... ..........  2% 6.32M 7s
   300K .......... .......... .......... .......... ..........  2% 15.9M 6s
   350K ....

  5850K .......... .......... .......... .......... .......... 49% 2.41M 1s
  5900K .......... .......... .......... .......... .......... 49% 6.75M 1s
  5950K .......... .......... .......... .......... .......... 50% 8.91M 1s
  6000K .......... .......... .......... .......... .......... 50% 2.37M 1s
  6050K .......... .......... .......... .......... .......... 50% 8.32M 1s
  6100K .......... .......... .......... .......... .......... 51% 6.72M 1s
  6150K .......... .......... .......... .......... .......... 51% 8.42M 1s
  6200K .......... .......... .......... .......... .......... 52% 2.47M 1s
  6250K .......... .......... .......... .......... .......... 52% 7.80M 1s
  6300K .......... .......... .......... .......... .......... 52% 6.80M 1s
  6350K .......... .......... .......... .......... .......... 53% 8.78M 1s
  6400K .......... .......... .......... .......... .......... 53% 2.35M 1s
  6450K .......... .......... .......... .......... .......... 54% 8.89M 1s
  6500K ....

 11650K .......... .......... .......... .......... .......... 97% 3.49M 0s
 11700K .......... .......... .......... .......... .......... 98% 3.04M 0s
 11750K .......... .......... .......... .......... .......... 98% 7.29M 0s
 11800K .......... .......... .......... .......... .......... 98% 3.25M 0s
 11850K .......... .......... .......... .......... .......... 99% 3.25M 0s
 11900K .......... .......... .......... .......... .......... 99% 6.54M 0s
 11950K .......... .......... .......... .                    100% 2.97M=2.7s

2025-02-09 20:37:38 (4.26 MB/s) - 'legacysurvey-1851p175-image-g.fits.fz' saved [12268800/12268800]

--2025-02-09 20:37:38--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/185/1853p175/legacysurvey-1853p175-image-g.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.113, 128.55.206.106, 128.55.206.107, ...
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.113|:443... connected.
HTTP request sent, awaiting respo

  4650K .......... .......... .......... .......... .......... 38% 1.05M 2s
  4700K .......... .......... .......... .......... .......... 39% 47.7M 2s
  4750K .......... .......... .......... .......... .......... 39% 67.7M 2s
  4800K .......... .......... .......... .......... .......... 40% 1.62M 2s
  4850K .......... .......... .......... .......... .......... 40% 5.45M 2s
  4900K .......... .......... .......... .......... .......... 41% 1.69M 2s
  4950K .......... .......... .......... .......... .......... 41% 1.78M 2s
  5000K .......... .......... .......... .......... .......... 41% 4.68M 2s
  5050K .......... .......... .......... .......... .......... 42% 1.51M 2s
  5100K .......... .......... .......... .......... .......... 42% 10.3M 2s
  5150K .......... .......... .......... .......... .......... 43% 1.50M 2s
  5200K .......... .......... .......... .......... .......... 43% 9.00M 2s
  5250K .......... .......... .......... .......... .......... 43% 1.47M 2s
  5300K ....

...

HTTP request sent, awaiting response... 200 OK
Length: 11301120 (11M)
Saving to: 'legacysurvey-1902p095-invvar-i.fits.fz'

     0K .......... .......... .......... .......... ..........  0%  674K 16s
    50K .......... .......... .......... .......... ..........  0% 1.34M 12s
   100K .......... .......... .......... .......... ..........  1% 11.0M 8s
   150K .......... .......... .......... .......... ..........  1%  730K 10s
   200K .......... .......... .......... .......... ..........  2% 90.3M 8s
   250K .......... .......... .......... .......... ..........  2% 32.6M 7s
   300K .......... .......... .......... .......... ..........  3% 6.55M 6s
   350K .......... .......... .......... .......... ..........  3% 1.43M 6s
   400K .......... .......... .......... .......... ..........  4% 1.48M 6s
   450K .......... .......... .......... .......... ..........  4% 1.45M 6s
   500K .......... .......... .......... .......... ..........  4% 6.78M 6s
   550K .......... .......... ........

  5300K .......... .......... .......... .......... .......... 48% 2.79M 3s
  5350K .......... .......... .......... .......... .......... 48% 1.56M 3s
  5400K .......... .......... .......... .......... .......... 49% 5.59M 3s
  5450K .......... .......... .......... .......... .......... 49% 1.62M 3s
  5500K .......... .......... .......... .......... .......... 50% 6.32M 3s
  5550K .......... .......... .......... .......... .......... 50% 1.59M 3s
  5600K .......... .......... .......... .......... .......... 51% 5.92M 3s
  5650K .......... .......... .......... .......... .......... 51% 1.61M 3s
  5700K .......... .......... .......... .......... .......... 52% 2.35M 3s
  5750K .......... .......... .......... .......... .......... 52% 2.76M 3s
  5800K .......... .......... .......... .......... .......... 53% 1.62M 3s
  5850K .......... .......... .......... .......... .......... 53% 5.92M 3s
  5900K .......... .......... .......... .......... .......... 53% 1.63M 3s
  5950K ....

Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11289600 (11M)
Saving to: 'legacysurvey-1905p095-invvar-i.fits.fz'

     0K .......... .......... .......... .......... ..........  0%  681K 16s
    50K .......... .......... .......... .......... ..........  0% 1.35M 12s
   100K .......... .......... .......... .......... ..........  1% 8.08M 8s
   150K .......... .......... .......... .......... ..........  1% 1.37M 8s
   200K .......... .......... .......... .......... ..........  2% 1.37M 8s
   250K .......... .......... .......... .......... ..........  2% 1.49M 8s
   300K .......... .......... .......... .......... ..........  3% 6.39M 7s
   350K .......... .......... .......... .......... ..........  3% 1.45M 7s
   400K .......... .......... .......... .......... ..........  4% 1.55M 7s
   450K .......... .......... .......... .......... ..........  4% 5.95M 6s
   500K .......... .......... .

  5750K .......... .......... .......... .......... .......... 52% 6.82M 1s
  5800K .......... .......... .......... .......... .......... 53% 5.52M 1s
  5850K .......... .......... .......... .......... .......... 53% 5.08M 1s
  5900K .......... .......... .......... .......... .......... 53% 5.47M 1s
  5950K .......... .......... .......... .......... .......... 54% 7.51M 1s
  6000K .......... .......... .......... .......... .......... 54% 5.91M 1s
  6050K .......... .......... .......... .......... .......... 55% 4.08M 1s
  6100K .......... .......... .......... .......... .......... 55% 6.99M 1s
  6150K .......... .......... .......... .......... .......... 56% 6.74M 1s
  6200K .......... .......... .......... .......... .......... 56% 6.30M 1s
  6250K .......... .......... .......... .......... .......... 57% 4.05M 1s
  6300K .......... .......... .......... .......... .......... 57% 7.25M 1s
  6350K .......... .......... .......... .......... .......... 58% 5.40M 1s
  6400K ....


mv tmpvar.fits u7854_ivar.fits
